# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
#!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [5]:
## Load in csv.gz
df = pd.read_csv('Data/Seattle-pizza.csv.gz', low_memory=False)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,343,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","['pickup', 'delivery']",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796
1,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113
2,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,964,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",['delivery'],$,"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405
3,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,205,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.598976, 'longitude': -122.294685}","['pickup', 'delivery']",$$,"{'address1': '2901 S Jackson St', 'address2': ...",1.206603e+10,(206) 602-6333,1571.867438
4,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],NaN,"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [6]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.6146934, 'longitude': -122.312..."
1,"{'latitude': 47.608127, 'longitude': -122.302435}"
2,"{'latitude': 47.615379179632, 'longitude': -12..."
3,"{'latitude': 47.598976, 'longitude': -122.294685}"
4,"{'latitude': 47.61849, 'longitude': -122.31664}"
...,...
835,"{'latitude': 47.6134491, 'longitude': -122.194..."
836,"{'latitude': 47.6161899, 'longitude': -122.351..."
837,"{'latitude': 47.6204816, 'longitude': -122.350..."
838,"{'latitude': 47.676215, 'longitude': -122.205084}"


- Why didn't that work???

In [13]:
## slice out a single test coordinate
test_coord = df['coordinates'][0]
#test_coord = df.loc[1,'coordinates']
test_coord


"{'latitude': 47.6146934, 'longitude': -122.3127642}"

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [14]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [15]:
## replace single ' with " 
test_coord = test_coord.replace("'",'"')
test_coord

'{"latitude": 47.6146934, "longitude": -122.3127642}'

In [16]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 47.6146934, 'longitude': -122.3127642}

### Now, how can we apply this same process to the entire column??

In [20]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'",'"')
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [21]:
## slice out a single test coordinate
test_coord = df.loc[0,'coordinates']
type(test_coord)

dict

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [22]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,latitude,longitude
0,47.614693,-122.312764
1,47.608127,-122.302435
2,47.615379,-122.323231
3,47.598976,-122.294685
4,47.618490,-122.316640
...,...,...
835,47.613449,-122.194542
836,47.616190,-122.351608
837,47.620482,-122.350445
838,47.676215,-122.205084


In [23]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df,df['coordinates'].apply(pd.Series)],axis = 1)
df = df.drop(columns = 'coordinates')
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,343,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['pickup', 'delivery']",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,47.614693,-122.312764
1,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,47.608127,-122.302435


## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [24]:
## Load in mapbox api credentials from .secret

###I DO NOT HAVE A MAPBOX JSON FILE

with open('/Users/craig/.secret/mapbox.json') as f:
    login = json.load(f)
login.keys()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/craig/.secret/mapbox.json'

- Use the plotly express `set_maptbox_acccess_token` function

In [ ]:
## set mapbox token
px.set_mapbox_access_token(login['api-key'])

In [ ]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df, lat='latitude',lon='longitude',mapbox_style='open-street-map')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df, lat='latitude',lon='longitude',mapbox_style='open-street-map', hover_name = 'name',
                  hover_data = ['price', 'rating', 'location']

### Fixing the Location Column

In [ ]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

> Also a string-dictionary...

In [ ]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

In [ ]:
## apply json.loads
df['location'] = df['location'].apply(json.loads)
df

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [ ]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

In [ ]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [ ]:
## test applying our function
df['location'].apply(fix_address)

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [ ]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)

In [ ]:
## filter for businesses with display_location == "ERROR"
errors = df[df['display_location']=='Error']

In [ ]:
## slice out a new test address and inspect
test_addr = df.loc[854, 'location']
test_addr

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [ ]:
## remove any rows where display_location == 'ERROR'
dr = df.loc[df['display_location']!='Error']

- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x :x['display_address'])
df.head(3)

- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [ ]:
## slice out a test_address
test_add = df.loc[339,'display_address']
test_add

In [ ]:
## test using .join with a "\n"
'\n'.join(test_add)

In [ ]:
## apply the join to every row with a lambda
df['Address'] = df['display_address'].apply(lambda x :x['display_address'])

### Final Map

In [ ]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat='latitude', lon='longitude',
                         mapbox_style='open-street-map', hover_name = 'name',
                         hover_data = ['price', 'rating', 'Address'], color = 'rating', 
                         title='Restaurants with Pizza in Seattle, WA')

pfig.show()

#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map

In [ ]:
## use fig.write_html to save map
pfig.write_html('final_map.html')

In [ ]:
#Click Widgets -> "Download Widget State" to keep the map present